In [ ]:
# crewai_example.py

from typing import List, Annotated
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent as crew_agent, crew as crew_crew, task as crew_task

from pydantic import BaseModel, Field
import networkx as nx
import os
import random
import dotenv

from protocol import SocialGraph, find_potential_peers

dotenv.load_dotenv()

# Mockup data
AGENTS_DATA = {
    "Agent1": {"links": ["Agent2", "Agent3", "Agent4"], "skills": ["Python", "Data Analysis"]},
    "Agent2": {"links": ["Agent1", "Agent3", "Agent5"], "skills": ["Java", "Machine Learning"]},
    "Agent3": {"links": ["Agent1", "Agent2", "Agent4"], "skills": ["Python", "NLP"]},
    "Agent4": {"links": ["Agent1", "Agent3", "Agent5"], "skills": ["C++", "Computer Vision"]},
    "Agent5": {"links": ["Agent2", "Agent4"], "skills": ["JavaScript", "Web Development"]},
}

# Initialize the social graph
social_graph = SocialGraph(AGENTS_DATA)

# Define the input model for the task
class FindPeersInput(BaseModel):
    task_description: str = Field(..., description="Description of the task")
    agent_id: str = Field(..., description="ID of the agent")
    search_depth: int = Field(2, description="Depth of search in the social graph")

# Define the output model for the task
class FindPeersOutput(BaseModel):
    potential_peers: List[str] = Field(..., description="List of potential peers")

# Define the agent configuration
AGENT_CONFIG = {
    "model": "gpt-3.5-turbo",
    "openai_api_key": os.getenv("OPENAI_API_KEY"),
    "temperature": 0,
}

@CrewBase
class PeerFindingCrew:
    """CrewAI implementation for finding potential peers"""

    # Since we are not using external config files, we'll define agents and tasks directly

    @crew_agent
    def peer_finder_agent(self) -> Agent:
        return Agent(
            name="peer_finder_agent",
            config=AGENT_CONFIG,
            verbose=True,
        )

    @crew_task
    def find_peers_task(self) -> Task:
        return Task(
            name="find_peers_task",
            agent=self.peer_finder_agent(),
            input_model=FindPeersInput,
            output_model=FindPeersOutput,
            prompt_template=self.find_peers_prompt(),
            tools=[],  # Add any tools if needed
            verbose=True,
        )

    def find_peers_prompt(self) -> str:
        return """
You are {agent_id}. Your task is to find potential peers for the given task.

Task Description: {task_description}

Use the provided function to find potential peers.

Provide the list of potential peers as a comma-separated string.

"""

    @crew_crew
    def crew(self) -> Crew:
        """Creates the PeerFinding crew"""
        return Crew(
            agents=self.agents,  # Automatically created by the @crew_agent decorator
            tasks=self.tasks,    # Automatically created by the @crew_task decorator
            process=Process.sequential,
            verbose=2,
        )

    def kickoff(self, inputs):
        # Prepare the input for the task
        task_input = FindPeersInput(
            task_description=inputs["task_description"],
            agent_id=inputs["agent_id"],
            search_depth=inputs.get("search_depth", 2),
        )

        # Run the task
        result = self.find_peers_task().run(task_input)

        # Output the result
        print(f"Potential peers for agent {inputs['agent_id']}: {', '.join(result.potential_peers)}")

# Usage
if __name__ == "__main__":
    # Set random seed for reproducibility
    random.seed(42)

    # Define the task and initial agent
    task_description = "Develop a machine learning model for natural language processing"
    initial_agent_id = "Agent1"

    # Initialize the crew
    crew_instance = PeerFindingCrew()

    # Run the crew with the specified inputs
    crew_instance.kickoff(
        inputs={
            "task_description": task_description,
            "agent_id": initial_agent_id,
            "search_depth": 2,
        }
    )
